In [10]:
import time

import numpy as np
import pandas as pd
import scipy.sparse as sparse
import torch, torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import Adam
from tqdm import tqdm_notebook

from code.layers import GCNConv
from code.utils import make_matrix, apply_weights, precision_at_k

In [5]:
df_train = pd.read_csv('prepared_data/ml-100k/train.csv')
df_test = pd.read_csv('prepared_data/ml-100k/test.csv')
df_test_warm = pd.read_csv('prepared_data/ml-100k/test_warm.csv')

df_train.head()

,user_id,item_id,feedback
0,259,255,4
1,259,286,4
2,259,298,4
3,259,185,4
4,259,173,4


In [6]:
matrix = make_matrix(df_train, count=False)
matrix = apply_weights(matrix, weight='bm25')

In [9]:
matrix = sparse.csr_matrix(matrix).tocsr()

In [ ]:
class GCNRecsys(object):
    
    def __init__(self, num_users, num_items, A, embedding_dim=64, sparse=True):
        super(GCNRecsys, self).__init__()
        
        # Usual nn.Embedding for users and items
        self.embedding_dim = embedding_dim
        self.user_embeddings = nn.Embedding(num_embeddings=num_users, 
                                            embedding_dim=embedding_dim,  
                                            sparse=sparse)
        self.item_embeddings = nn.Embedding(num_embeddings=num_items, 
                                            embedding_dim=embedding_dim,  
                                            sparse=sparse)
        self.user_biases = ZeroEmbedding(num_users, 1, sparse=sparse)
        self.item_biases = ZeroEmbedding(num_items, 1, sparse=sparse)
        
        # Secind layer only for items: embedding_dim -> GCN -> embedding_dim
        self.item_gcn = GCNConv(A=A, 
                                in_channels=embedding_dim, 
                                out_channels=embedding_dim)
        
    def forward(self, user_ids, item_ids):
        